<a href="https://colab.research.google.com/github/stanislavprihoda/python-and-ML-courses/blob/master/FastAI/dl2/06_Cuda_CNN_hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('//content/gdrive/My Drive/Colab Notebooks/')

In [0]:
#export
from exp.nb_05b import *
torch.set_num_threads(2)

### ConvNet

In [0]:
x_train,y_train,x_valid,y_valid = get_data()

In [0]:
#export
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [0]:
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [6]:
x_train.mean(),x_train.std()

(tensor(3.0614e-05), tensor(1.))

In [0]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [0]:
#export
class Lambda(nn.Module):
    def __init__(self,func):
        super().__init__()
        self.func = func
    def forward(self,x): return self.func(x)
    
def flatten(x): return x.view(x.shape[0],-1)

In [0]:
def mnist_resize(x): return x.view(-1,1,28,28)

In [0]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d(1,8,5,padding=2,stride=2),nn.ReLU(), #14
        nn.Conv2d(8,16,3,padding=1,stride=2),nn.ReLU(), #7
        nn.Conv2d(16,32,3,padding=1,stride=2),nn.ReLU(),#4
        nn.Conv2d(32,32,3,padding=1,stride=2),nn.ReLU(),#2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,data.c)
    )

In [0]:
model = get_cnn_model(data)

In [0]:
cbfs=[Recorder,partial(AvgStatsCallback,accuracy)]

In [0]:
opt=optim.SGD(model.parameters(),lr=0.4)
learn=Learner(model,opt,loss_func,data)
run=Runner(cb_funcs=cbfs)

In [14]:
%time run.fit(1,learn)

train: [1.8596523324238885, tensor(0.3487)]
valid: [0.676917626953125, tensor(0.7898)]
CPU times: user 5.92 s, sys: 337 ms, total: 6.26 s
Wall time: 3.99 s


### CUDA

In [0]:
#more flexible way
device = torch.device('cuda',0)

In [0]:
class CudaCallback(Callback):
    def __init__(self,device): self.device=device
    def begin_fit(self): self.model.to(device)
    def begin_batch(self):self.run.xb,self.run.yb=self.xb.to(device),self.yb.to(device)

In [0]:
#less flexible but quite convenient
torch.cuda.set_device(device)

In [24]:
def cuda_check():
  if not hasattr(torch._C, '_cuda_setDevice'):
      print('CUDA not available. Setting device=-1.')
      cuda_id = -1
      torch.cuda.set_device(cuda_id)
  else:
      print('Setting CUDA.')
      device = torch.device('cuda',0)
      torch.cuda.set_device(device)
cuda_check()

Setting CUDA.


In [0]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [0]:
cbfs.append(CudaCallback)

In [0]:
model = get_cnn_model(data)

In [0]:
opt = optim.SGD(model.parameters(),lr=0.4)
learn = Learner(model,opt,loss_func,data)
run=Runner(cb_funcs=cbfs)

In [23]:
%time run.fit(3,learn)

train: [2.3180968294438626, tensor(0.1391, device='cuda:0')]
valid: [2.2868765625, tensor(0.1064, device='cuda:0')]
train: [1.4699159602528995, tensor(0.4907, device='cuda:0')]
valid: [0.46128642578125, tensor(0.8584, device='cuda:0')]
train: [0.32717124703004186, tensor(0.9012, device='cuda:0')]
valid: [0.16977301025390626, tensor(0.9502, device='cuda:0')]
CPU times: user 4.91 s, sys: 1.51 s, total: 6.42 s
Wall time: 9.05 s


### Model refactor

In [0]:
def conv2d(ni, nf, ks=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride), nn.ReLU())

In [0]:
#export
class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def begin_batch(self): self.run.xb = self.tfm(self.xb)

def view_tfm(*size):
    def _inner(x): return x.view(*((-1,)+size))
    return _inner

In [0]:
mnist_view = view_tfm(1,28,28)
cbfs.append(partial(BatchTransformXCallback, mnist_view))

In [0]:
nfs=[8,16,32,32]

In [0]:
def get_cnn_layers(data,nfs):
  nfs = [1] + nfs
  return [
      conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3)
      for i in range(len(nfs)-1)
  ] + [nn.AdaptiveAvgPool2d(1),Lambda(flatten),nn.Linear(nfs[-1], data.c)]

def get_cnn_model(data,nfs): return nn.Sequential(*get_cnn_layers(data,nfs))

In [0]:
#export
def get_runner(model, data, lr=0.6, cbs=None, opt_func=None, loss_func = F.cross_entropy):
    if opt_func is None: opt_func = optim.SGD
    opt = opt_func(model.parameters(), lr=lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner(cb_funcs=listify(cbs))

In [0]:
model = get_cnn_model(data,nfs)
learn,run = get_runner(model, data, lr=0.4,cbs=cbfs)

In [35]:
model

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (4): AdaptiveAvgPool2d(output_size=1)
  (5): Lambda()
  (6): Linear(in_features=32, out_features=10, bias=True)
)

In [36]:
run.fit(3,learn)

train: [2.248745161233489, tensor(0.1644, device='cuda:0')]
valid: [1.60773525390625, tensor(0.3607, device='cuda:0')]
train: [0.8686363770789707, tensor(0.7080, device='cuda:0')]
valid: [0.2608685302734375, tensor(0.9206, device='cuda:0')]
train: [0.21865404266672037, tensor(0.9329, device='cuda:0')]
valid: [0.1367408935546875, tensor(0.9604, device='cuda:0')]


In [0]:
nn.Conv2d??